In [ ]:
from flask import Flask, render_template, request, redirect

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    file = request.files['file_name'] # from post request: from all files get file 'file_name'
    file.save(f'uploads/img.jpg')
    return redirect('/')


if __name__ == "__main__":
    app.run()

In [22]:
from flask import Flask, render_template, request, redirect, send_from_directory
import os
from werkzeug.utils import secure_filename
from werkzeug.exceptions import RequestEntityTooLarge

app = Flask(__name__, template_folder="templates/")
app.config['UPLOAD_DIRECTORY'] = 'uploads/'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB
app.config['ALLOWED_EXTENSIONS'] = ['.jpg', '.jpeg', '.png', '.gif']

@app.route('/')
def index():
    files = os.listdir(app.config['UPLOAD_DIRECTORY'])
    images = []
    for filename in files:
        extension = os.path.splitext(filename)[1].lower()
        if extension in app.config['ALLOWED_EXTENSIONS']:
            images.append(filename)

    return render_template('index.html', images=images)

@app.route('/upload', methods=['POST'])
def upload():
    try:
        file = request.files['file_name']
        extension = os.path.splitext(file.filename)[1].lower()

        if file and extension in app.config['ALLOWED_EXTENSIONS']:
            file.save(os.path.join(
                app.config['UPLOAD_DIRECTORY'],
                secure_filename(file.filename)
            ))
        else:
            return 'File is not an allowed image type.'

    except RequestEntityTooLarge:
        return 'File is larger than the 16MB limit.'

    return redirect('/')

@app.route('/serve-image/<filename>', methods=['GET'])
def serve_image(filename):
    return send_from_directory(app.config['UPLOAD_DIRECTORY'], filename)

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Jan/2024 22:44:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2024 22:44:18] "GET /static/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [06/Jan/2024 22:44:18] "GET /serve-image/img_out.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [06/Jan/2024 22:44:18] "GET /serve-image/img.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [06/Jan/2024 22:44:18] "GET /serve-image/Routine.png HTTP/1.1" 304 -
127.0.0.1 - - [06/Jan/2024 22:44:19] "GET /favicon.ico HTTP/1.1" 404 -
